# Create Table from instance segmentation masks

This notebook demonstrates how to create a table of instance segmentation masks.

"Instance segmentation", as opposed to "semantic segmentation", refers to situations where each "object" in your dataset has some associated segmentation mask which might overlap with other objects. "Semantic segmentation", by contrast, refers to situations where each pixel in each image of your dataset is associated with exactly one class label. See [this notebook](https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/1-create-tables/create-semantic-segmentation-dataset.ipynb) for an example of how to create a table from semantic segmentation masks.

This notebook will use instance segmentations represented as a 3D tensor of shape `(width, height, num_instances)`. For an example of how to use instance segmentations represented as polygons, see [this notebook](https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/1-create-tables/create-instance-segmentation-polygons-table.ipynb).

We will use the COCO128 for this example. Note that while 3LC has built-in functionality to facilitate the loading of COCO-style datasets, this example will more "manually" load the dataset to demonstrate how this can be done in general.

## Project Setup

In [ ]:
PROJECT_NAME = "3LC Tutorials"
DATASET_NAME = "COCO128 Segmentation Masks"

## Imports

In [ ]:
import json
from pathlib import Path

import numpy as np
from pycocotools import mask as mask_utils

import tlc

## Get images and masks

In [ ]:
coco128_path = Path("../../data/coco128")

# Open COCO128 annotations.json file and load the annotations
with open(coco128_path / "annotations.json") as f:
    annotations = json.load(f)

# Get the images, segmentations and classes from the data file
images = annotations["images"]
instances = annotations["annotations"]
classes = [category["name"] for category in annotations["categories"]]

We want to create a list of rows with the following structure:

```
{
    "image": str,
    "segmentation: {
        "image_width": int,
        "image_height": int,
        "instance_properties": {
            "label": list[int],
        },
        "masks": np.ndarray
    }
}
```

Where `image` is a path to the image file, and `segmentation` is a dict of the format expected by 3LC for polygon-based instance segmentation. Its fields are:

- `image_width`: The width of the image
- `image_height`: The height of the image
- `instance_properties`: A dict of properties for each instance in the segmentation. The keys are the names of the properties, and the values are a list containing the value of that property for each instance in the image. Examples of instance properties include `label` and `confidence`.
- `masks`: A numpy array of shape `(width, height, num_instances)` and dtype `uint8`, where each slice along the third dimension is a binary mask for one of the instances in the image. This specific format (order of dimensions and dtype) was chosen to match the format used by pycocotools, which is used for 3LCs handling of all instance segmentation data.

We will now create a list of rows with the above structure.

In [ ]:
# Create a list of rows, holding information about each image
# The instance properties are initially empty, and will be populated later
rows = [
    {
        "image": (coco128_path / "images" / image["file_name"]).absolute().as_posix(),
        "segmentation": {
            "image_width": image["width"],
            "image_height": image["height"],
            "instance_properties": {
                "label": [],
            },
            "masks": np.empty((image["width"], image["height"], 0), dtype=np.uint8),
        },
    }
    for image in images
]

In [ ]:
# Create a mapping from image index to list of polygons
polygons = {i: [] for i in range(len(images))}

for instance in instances:
    # Get the row index for the instance
    row_id = instance["image_id"]

    # Get the label and polygon for the instance
    label = instance["category_id"]
    polygon = instance["segmentation"][0]

    # Add the label to the row
    rows[row_id]["segmentation"]["instance_properties"]["label"].append(label)

    # Add the polygon to the list of polygons for the image
    polygons[row_id].append(polygon)

### Convert the polygons to masks

Since the instance segmentation data in COCO128 is represented as polygons, we need to convert the polygons to masks for the purposes of this example. If you are creating a Table from your own instance segmentation mask data, the only thing you need to ensure is that the format of the `masks` field matches the format expected by 3LC.

Here, we use pycocotools to convert the polygons to masks.

In [ ]:
# Iterate over all the images and convert the polygons to masks
for row_id, instance_polygons in polygons.items():
    # If there are no polygons for this image, create an empty mask
    if len(instance_polygons) == 0:
        rows[row_id]["segmentation"]["masks"] = np.empty(
            (images[row_id]["width"], images[row_id]["height"], 0), dtype=np.uint8
        )
        continue

    # Convert the polygons to a mask
    rles = mask_utils.frPyObjects(instance_polygons, images[row_id]["height"], images[row_id]["width"])
    masks = mask_utils.decode(rles)
    rows[row_id]["segmentation"]["masks"] = masks

## Create a table from the rows

We will now use a `TableWriter` to write the rows to a `Table`.

In [ ]:
# Define the column schemas of our Table
column_schemas = {
    "image": tlc.ImagePath,
    "segmentation": tlc.InstanceSegmentationMasks(
        name="segmentation", instance_properties_structure={"label": tlc.CategoricalLabel("label", classes=classes)}
    ),
}

In [ ]:
# Create a TableWriter
table_writer = tlc.TableWriter(
    table_name="initial",
    dataset_name=DATASET_NAME,
    project_name=PROJECT_NAME,
    column_schemas=column_schemas,
    if_exists="rename",
)

In [ ]:
# Write the rows to the Table
for row in rows:
    table_writer.add_row(row)

# Once we are done, call `finalize()` to create the Table
table = table_writer.finalize()

The table will now be visible in the 3LC Dashboard and can be used in any of your scripts or notebooks.